In [73]:
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [67]:
no_use_list=[]
'b2b','sharing_type','has_previous_accident','car_model','accident_ratio','socarpass','socarsave','repair_cnt','insurance_site_aid_YN','total_prsn_cnt','car_part2','insurance_site_aid_YN'
num_col_list=['repair_cost','insure_cost']

no_cat_list = num_col_list + ['fraud_YN', 'test_set']

In [58]:
no_cat_list

['repair_cost', 'insure_cost', 'fraud_YN', 'test_set']

In [68]:
one_hot_col_list= [col for col in data.columns if col not in no_cat_list]
OHE = ce.OneHotEncoder(cols=[col for col in data.columns if col not in no_cat_list])
data = OHE.fit_transform(data)

In [56]:
scaler= RobustScaler()
data[num_col_list]=scaler.fit_transform(data[num_col_list])

In [64]:
test_y, train_y=data["fraud_YN"][data['test_set']==1],data["fraud_YN"][data['test_set']==0]
test_x, train_x=data[data['test_set']==1].drop(["fraud_YN",'test_set'], axis=1),data[(data['test_set']==0)].drop(["fraud_YN",'test_set'], axis=1)

In [65]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((12879, 107), (3121, 107), (12879,), (3121,))

In [ ]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score)
def get_clf_eval(y_test, pred):
    acc=accuracy_score(y_test, pred)
    pre=precision_score(y_test, pred)
    re=recall_score(y_test, pred)
    f1=f1_score(y_test, pred)
    auc=roc_auc_score(y_test, pred)
    return acc,pre,re,f1,auc

In [ ]:
from sklearn.metrics import confusion_matrix
def print_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test, pred)
    acc, pre, re, f1, auc = get_clf_eval(y_test, pred)
    
    print(confusion)
    print(acc,pre)
    print(re,f1,auc)

In [70]:
pre_process = ColumnTransformer(remainder = 'passthrough', 
                                transformers=[('drop_columns', 'drop',no_use_list),
                               ('scaler',scaler,num_col_list)])

In [74]:
full_pipeline = Pipeline(steps=[('pre_processing', pre_process), ('random_forest', RandomForestClassifier(max_depth=10,random_state=2))])

In [76]:
full_pipeline.fit(train_x, train_y)

Pipeline(steps=[('pre_processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_columns', 'drop', []),
                                                 ('scaler', RobustScaler(),
                                                  ['repair_cost',
                                                   'insure_cost'])])),
                ('random_forest',
                 RandomForestClassifier(max_depth=10, random_state=2))])

In [77]:
full_pipeline.predict(test_x)

array([0, 0, 0, ..., 0, 0, 0])

In [13]:
# chat('police_site_aid_YN')

In [ ]:
# def scaling(x):
#     data_num_tmp=data_num.copy()
#     transformer = x()
#     x_data = transformer.fit_transform(data_num_tmp)
#     data_num_tmp[data_num_tmp.columns]=x_data
#     pd.plotting.scatter_matrix(data_num_tmp, alpha=0.2)
#     data_num_tmp['fraud']=data['fraud_YN']
#     insure_cost_dist=data_num_tmp[data_num_tmp['fraud']==1][['repair_cost','insure_cost']]
#     insure_cost_dist.plot.box()
# for i in [RobustScaler, StandardScaler, MinMaxScaler]:
#     scaling(i)

In [ ]:
y_test, y_train=data[["fraud_YN"]][data['test_set']==1],data[["fraud_YN"]][data['test_set']==0]
x_test, x_train=data[data['test_set']==1].drop("fraud_YN", axis=1),data[(data['test_set']==0)].drop("fraud_YN", axis=1)

In [ ]:
x_train

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import RobustScaler
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OrdinalEncoder
# from sklearn.preprocessing import OneHotEncoder

# num_col_list=['repair_cost','insure_cost']
# drop_list= num_col_list + ['fraud_YN']
# cat_col_list=[x for x in data.columns if x not in drop_list]

# num_pipeline = Pipeline([
#     ('robust_scaler', RobustScaler())
# ])

# full_pipeline = ColumnTransformer([
#     ("num", num_pipeline, num_col_list),
#     ("cat", OneHotEncoder(),cat_col_list)
# ])
# prepared_data = full_pipeline.fit_transform(x_train)

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(prepared_data, y_train)

In [ ]:
def get_clf_eval(y_test, pred):
    acc=accuracy_score(y_test, pred)
    pre=precision_score(y_test, pred)
    re=recall_score(y_test, pred)
    f1=f1_score(y_test, pred)
    auc=roc_auc_score(y_test, pred)
    return acc,pre,re,f1,auc